In [ ]:
from __future__ import unicode_literals, print_function, division

from Model import *
from train import *

# load file

from torch.utils.data import Dataset, DataLoader,SubsetRandomSampler
from data_loader import *
import torch
torch.cuda.set_device(0)

def ps_labeling(data_loader, model, semi_label, threshold,  dropout,rp = 2):
    #semi_label = torch.tensor(semi_label, dtype=torch.long).to(device)
    semi_label = torch.zeros(rp, len(semi_label), 60).to(device)
    softmax = nn.Softmax(dim=1)

    for iter_drop in range(rp):

        for it, (data, seq_len, label, _, index) in enumerate(data_loader):
            if it == 0:
                pass
            data = data.to(device)

            cla_pre_tmp = model(data, seq_len).detach()
            cla_pre_tmp = softmax(cla_pre_tmp)
            semi_label[iter_drop, index, :] = cla_pre_tmp
            break
            # index = torch.tensor(index).to(device)
            # sub_set, ps_lab = torch.max(cla_pre, 1)
            # sub_set = sub_set > 0
            # ps_lab = ps_lab +1

    np.save('./labels/ps_prob_full_rp%d_drop%.2f.npy'% (rp, dropout), semi_label.cpu().numpy())

## training procedure
teacher_force = False
fix_weight = True
fix_state = False
few_knn = False
max_length = 50
pro_tr = 0
pro_re = 0
phase  = 'RC'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# global variable
ProjectFolderName = 'NTUProject'
root_path = '/home/ws2/Documents/jingyuan/'
train_data = 'NTUtrain_cs_full.h5'
test_data = 'NTUtest_cs_full.h5'
# hyperparameter
feature_length = 75
hidden_size = 1024
batch_size = 64
en_num_layers = 3
de_num_layers = 1
middle_size = 125
cla_num_layers = 1
learning_rate = 0.001
epoch = 300
cla_dim = [60]

k = 2 # top k accuracy
# for classification
dataset_train = MyInterTrainDataset(os.path.join(root_path, ProjectFolderName, train_data), [])
semi_label = np.load('./labels/base_semiLabel.npy')#np.zeros(len(dataset_train))
indices_train = np.where(semi_label == 0)[0]
train_sampler = SubsetRandomSampler(indices_train)

train_loader = torch.utils.data.DataLoader(dataset_train, batch_size=batch_size,
                           shuffle=False, collate_fn=pad_collate_iter)



for drop_out in [0.3]:
    model = SemiSeq(feature_length, hidden_size, cla_dim, dropout=drop_out).to(device)
    optimizer= optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=learning_rate)
    model_name = './trained_model/' + 'baselineAP5_layer3_hid1024_epoch193'
    model, _ = load_model(model_name, model, optimizer, device)
    model.train()
    threshold = 0.2
    ps_labeling(train_loader, model, semi_label, threshold, dropout=drop_out)